## ASSIGNMENT #9  - Calculate Nitrogen deposition across europe to help farmers optimize fertilizer application (Real World)
- Sync your Fork


### 9.1 - Intro
- Over fertilization or farmland can lead to groundwater pollution. The European Farmers Association has requested you to build a program that provides the ammount of nitrogen atmospheric deposition across europe, for any particular location in Europe. This will allow farmers to optimize (reduce) the ammount of fertilizer application, reducing groundwater pollution while also reducing costs
- the python program will receive a coordinate (decimal lat and lon), and outputs a json with the Nitrogen deposition in Kg/ha and some metadata

### 9.2 - Mission
- <span style="color:green">Create a file called ***ndeposition.py*** in your FORGE</span> or <span style="color:green">Copy this notebook file into your forge and continue coding in this file</span>
- Consider the file ***EMEP01_rv4.42_year.2019met_2019emis.nc***
- Total nitrogen deposition is the sum of wet and dry, reduced and oxidized forms of Nitrogen:
  - Consider our file EMEP01_rv4.42_year.2019met_2019emis.nc
  - Look at the metadata @ https://emep.int/mscw/mscw_moddata.html to check which are the name of the layers in the NetCDF for the dry and wet, oxidized and reduced nitrogen
  
  - Retrieve those, sum them up and convert it to Kg/ha, for the location near Beja, with Lat: 37.9, Lon: -7.8
-
- The output should be the following json:

```json
{
    "version": "0.1",
    "version_notes": "Exercise for Green DataScience. Data extracted from 2019 EMEP (www.emep.int) dataset EMEP01_rv4.42_year.2019met_2019emis.nc with 0.1 degrees resolution (approx 11.1 km) - Dry and wet deposition of oxidized and reduced N (DDEP_OXN_m2Grid + DDEP_RDN_m2Grid + WDEP_OXN + WDEP_RDN) converted from mgN m-2 to kg ha-1",
    "coordinate_requested": {
        "lon": "-7.80",
        "lat": "37.90"
    },
    "coordinate_retrieved": {
        "lon": "-7.85",
        "lat": "37.95"
    },
    "data": {
        "total_n_deposition": {
            "value": "2.6",
            "unit": "kg ha-1"
        }
    }
}
```

- Wet deposition of oxidized nitrogen --- WDEP_OXN
- Wet deposition of reduced nitrogen --- WDEP_RDN
- Dry deposition of oxidized nitrogen per m2 grid --- DDEP_OXN_m2Grid
- Dry deposition of reduced nitrogen per m2 grid --- DDEP_RDN_m2Grid
https://emep.int/publ/reports/2019/EMEP_Status_Report_1_2019.pdf:
"For [...] **nitrogen compounds**, the **basic units** used throughout this report are μg (S or
N)/m3 for air concentrations and **mg (S or N)/m2 for depositions**"

In [22]:
# Load your NetCDF as before:
import netCDF4 as nc 
import numpy as np
import json
fn = 'C:/Users/johan/greends/greends-ipython/EMEP01_rv4.42_year.2019met_2019emis.nc' # update this path to your own file

ds = nc.Dataset(fn)

#list of the lat, lon, coordinates and precipitation
lats = ds.variables['lat'][:]
lons = ds.variables['lon'][:]
dry_NOXdep = ds.variables['DDEP_OXN_m2Grid']
dry_NRDdep = ds.variables['DDEP_RDN_m2Grid']
wet_NOXdep = ds.variables['WDEP_OXN']
wet_NRDdep = ds.variables['WDEP_RDN']

# Retrieve indexes of coordinates values for the location near Beja, with Lat: 37.9, Lon: -7.8
inLat = 37.9
inLon = -7.8
indexLat = np.absolute(lats-inLat).argmin()
indexLon = np.absolute(lons-inLon).argmin()

# retrieve values
outLat = round(lats[indexLat],2)
outLon = round(lons[indexLon],2)
dNox = dry_NOXdep[:,indexLat, indexLon]
dNrd = dry_NRDdep[:,indexLat, indexLon]
wNox = wet_NOXdep[:,indexLat, indexLon]
wNrd = wet_NRDdep[:,indexLat, indexLon]

# sum them up and convert it to Kg/ha from mg/m2
total_N_dep = dNox + dNrd + dNox+ wNrd
total_N_dep = round(float(total_N_dep) * 0.01,2)

# generate dict
out = {
    "version": "0.1",
    "version_notes": "Exercise for Green DataScience. Data extracted from 2019 EMEP (www.emep.int) dataset EMEP01_rv4.42_year.2019met_2019emis.nc with 0.1 degrees resolution (approx 11.1 km) - Dry and wet deposition of oxidized and reduced N (DDEP_OXN_m2Grid + DDEP_RDN_m2Grid + WDEP_OXN + WDEP_RDN) converted from mgN m-2 to kg ha-1",
    "coordinate_requested": {
        "lon": inLon,
        "lat": inLat
    },
    "coordinate_retrieved": {
        "lon": outLon,
        "lat": outLat
    },
    "data": {
        "total_n_deposition": {
            "value": total_N_dep,
            "unit": "kg ha-1"
        }
    }
}

# convert to json file
out = json.dumps(out, indent=4)
print(out)

{
    "version": "0.1",
    "version_notes": "Exercise for Green DataScience. Data extracted from 2019 EMEP (www.emep.int) dataset EMEP01_rv4.42_year.2019met_2019emis.nc with 0.1 degrees resolution (approx 11.1 km) - Dry and wet deposition of oxidized and reduced N (DDEP_OXN_m2Grid + DDEP_RDN_m2Grid + WDEP_OXN + WDEP_RDN) converted from mgN m-2 to kg ha-1",
    "coordinate_requested": {
        "lon": -7.8,
        "lat": 37.9
    },
    "coordinate_retrieved": {
        "lon": -7.85,
        "lat": 37.85
    },
    "data": {
        "total_n_deposition": {
            "value": 2.92,
            "unit": "kg ha-1"
        }
    }
}
